In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from tabulate import tabulate

In [ ]:
# Load the dataset
df = pd.read_csv("Data/data set.csv")

In [ ]:
# Load a copy of the dataset
df_index = df.copy()

# Delete columns
df_index.drop(['CONS_NO','10/3/2014'],axis=1,inplace=True)

In [ ]:
# Remove data from the majority class (FLAG = 0)
df_clean = df_index.copy()
df_minority = df_clean[df_clean['FLAG'] == 1]
df_majority = df_clean[df_clean['FLAG'] == 0].sample(len(df_minority))
df_undersampling = pd.concat([df_minority, df_majority]).reset_index()

In [ ]:
# Load a copy of the dataset
df_index = df_undersampling.copy()

In [ ]:
# Delete columns
df_index.drop(['index'],axis=1,inplace=True)

In [ ]:
# Load a copy of the dataset
df_filtered = df_index.copy()

In [ ]:
# Drop the target column that has no null values to perform imputation
df_filtered.drop(['FLAG'],axis=1,inplace=True)

In [ ]:
# Impute the data with respect to 5 neighbors
imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(df_filtered)
imputed.shape

In [ ]:
# Create a new dataset with the imputed data
df_imputed = pd.DataFrame(imputed, columns=df_filtered.columns)

In [ ]:
# Copy of the inputed dataset
df_low = df_imputed.copy()

In [ ]:
# Add the FLAG column again
df_low['FLAG'] = df_undersampling['FLAG']

In [ ]:
# Splitting the data from the label
X = df_low.drop(['FLAG'], axis=1)
y = df_low['FLAG']

In [ ]:
# Splitting the training data from the testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize a Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)

# Define the parameter grid to explore
param_grid = {'n_estimators': [50],'max_depth': [5],'min_samples_split': [2]}

# Initialize the GridSearchCV object for Gradient Boosting
grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid, scoring='recall', cv=3, n_jobs=-1)

# Fit the model to the training data
grid_search_gb.fit(X_train, y_train)

y_pred_gb = best_gb.predict(X_test)